# Input Perterbation

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt




import pandas as pd

with open('ua.base', 'r') as file:
    # Read the lines of the file
    lines = file.readlines()

# Process the lines to create a list of tuples
data = []
for line in lines:
    parts = line.strip().split('\t')
    user_id = int(parts[0])
    item_id = int(parts[1])
    rating = int(parts[2])
    # Skip the timestamp (parts[3])
    data.append((user_id, item_id, rating))

# Create a DataFrame from the list of tuples
df = pd.DataFrame(data, columns=['userId', 'movieId', 'rating'])

# Print the first few rows of the DataFrame

df['movieId'].max()
df['movieId'].min()

temp = 1
userItem  = np.zeros((df['userId'].max()+1,df['movieId'].max()+1))
print(userItem )
print(userItem.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(944, 1683)


In [ ]:
for i in range (len(df['userId'])):
  userItem[df.userId[i]][df.movieId[i]] = df.rating[i]

print(userItem)
print(userItem.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 5. 3. ... 0. 0. 0.]
 [0. 4. 0. ... 0. 0. 0.]
 ...
 [0. 5. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 5. ... 0. 0. 0.]]
(944, 1683)


In [ ]:
R = userItem[1:]
print(R)
R = R[:, 1:]
print(R)

ep = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]

[[0. 5. 3. ... 0. 0. 0.]
 [0. 4. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 5. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 5. ... 0. 0. 0.]]
[[5. 3. 4. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]]


In [ ]:
A = R>0.5; A[A == True] = 1; A[A == False]   = 0; A = A.astype(np.float64, copy=False)


In [ ]:
def input_perterbation_sgd(R,A, d, gamma, lambd, epsilon):
    k = 50
    num_users, num_items = R.shape
    P = np.random.rand(num_users, d)
    Q = np.random.rand(num_items, d)
    R1 = R + np.random.laplace(scale=(4) / epsilon, size=R.shape)
    R0 = np.zeros((R.shape))
    B =3
    for i in range(R.shape[0]):
        for j in range (R.shape[1]):
            if A[i,j] != 0: R0[i,j ] = R1[i,j]


    R0 = np.clip(R0, -B, B)
    # Assuming R is a numpy array, find non-zero (i.e., rated) entries
    rated_indices = np.argwhere(R > 0)

    for epoch in range(k):
        np.random.shuffle(rated_indices)  # Shuffle the observed ratings
        for i, j in rated_indices:
            e_ui = R[i, j] - np.dot(P[i], Q[j])


            # Update rule remains the same, but only applied to observed ratings
            P[i] += gamma * (e_ui * Q[j] - lambd * P[i])
            Q[j] += gamma * (e_ui * P[i] - lambd * Q[j])

    return P, Q , P.dot(Q.T)


In [ ]:
p,q = input_perterbation_sgd(R,A, 3,0.01,0.6,  epsilon = 2  )

In [ ]:
print(p.dot(q.T))

r= p.dot(q.T)


[[3.24148069 2.755752   2.58690929 ... 1.56275807 2.59891697 2.45790424]
 [3.39235033 2.88401213 2.70731602 ... 1.63456665 2.72069755 2.57350531]
 [2.68354203 2.2814171  2.1416417  ... 1.29280829 2.15242854 2.03608814]
 ...
 [3.45249414 2.93514568 2.75530953 ... 1.66442624 2.76781877 2.61709932]
 [3.65775141 3.1096441  2.91912271 ... 1.76286219 2.9333022  2.77461298]
 [3.25984308 2.77136192 2.60156549 ... 1.57122627 2.61406164 2.47254506]]


In [ ]:
import math
def RMSE(actual, predicted):
    err = 0

    count = np.count_nonzero(actual)
    for i in range(actual.shape[0]):
        for j in range(actual.shape[1]):
            if actual[i][j] > 0:

                err += (actual[i][j] - predicted[i][j])**2

    if count == 0: "error is not calculated therefore count = 0 "
    else :
        mse = err/count
    rmse = math.sqrt(mse)

    return  rmse

l =RMSE(R,r)
print(l)

1.0968061661912412


In [ ]:
ISGD_Rmse = []
ep = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
for _ in (ep):

    p,q , r = input_perterbation_sgd(R,A, 3,0.01,0.3,  epsilon = _  )

    temp = RMSE(R,r)
    ISGD_Rmse.append(temp)

In [ ]:
print(ISGD_Rmse)

[0.9699181823985162, 0.9703882077044553, 0.9660848300554666, 0.9710855341136666, 0.9712712229249982, 0.9672139675597393, 0.9700329480374184, 0.9702701381382745, 0.9659381463475186, 0.9685991747796134]
